In [792]:
### Analyze MOFA MODEL Results
### Investigate the specific top features of each factor
### Plot the normalized expression of the features as heatmap

#############################################
# Prerequisites - Load Libraries

In [793]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/mofa_analysis//lib/R/library"


In [794]:
source('MS2_Plot_Config.r')

In [795]:
source('MS1_Functions.r')

###############################################
# Preqrequisites Configurations & Parameters

In [796]:
### Load the parameters that are set via the configuration files

In [797]:
### Load configurations file
global_configs = read.csv('configurations/Data_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/Data_Configs.csv'”


In [798]:
head(global_configs,2)

,parameter,value
,<chr>,<chr>
1,data_path,/lustre/groups/epigenereg01/workspace/projects/jove/example_data/
2,result_path,/lustre/groups/epigenereg01/workspace/projects/jove/example_results/


In [799]:
data_path = global_configs$value[global_configs$parameter == 'data_path']

In [800]:
data_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/example_data/"

In [801]:
result_path = global_configs$value[global_configs$parameter == 'result_path']

In [802]:
result_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/example_results/"

In [803]:
### Configuration parameters for plot (read and assign)

In [804]:
plot_configs =  read.csv('configurations/05_Feature_Analysis_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/05_Feature_Analysis_Configs.csv'”


In [805]:
head(plot_configs,2)

,configuration_name,mofa_result_name,factor,top_variable_thres,faceting_variable,type,plot_width,plot_height
,<chr>,<chr>,<chr>,<dbl>,<chr>,<lgl>,<dbl>,<int>
1,MI_v1,MI_v1_MOFA,Factor2,0.005,disease,NA,8.07,5


In [806]:
## Assign the values from the configuration file to the corresponding variables

In [807]:
mofa_name = plot_configs$mofa_result_name
data_name = plot_configs$configuration_name

factor_var = plot_configs$factor
top_variable_fraction = plot_configs$top_variable_thres
variable = plot_configs$faceting_variable

view_var = plot_configs$type

In [808]:
## Extract the specified plot size
width_par = plot_configs$plot_width  # 8.07 default
height_par = plot_configs$plot_height

In [809]:
### Alternative: Set configurations manually 

# Load Data 

## MOFA Input

In [810]:
### Load the normalized expression data that was used as input for the MOFA model

In [811]:
path = paste0(result_path, '/02_results/02_Combined_Data_',data_name,'_INTEGRATED.csv')
data_long = read.csv(path)
data_long$X = NULL
print(file.info(path)$mtime)

[1] "2024-05-10 22:27:00 CEST"


In [812]:
head(data_long,2)

,sample_id,variable,value,type,gene
,<chr>,<chr>,<dbl>,<chr>,<chr>
1,D1,adipocyte__ABI1,-1.1243382,adipocyte,ABI1
2,D2,adipocyte__ABI1,-0.8572543,adipocyte,ABI1


## MOFA Model

In [813]:
### Load the trained MOFA Model

In [814]:
model_name =  paste0("03_MOFA_MODEL_", mofa_name,'.hdf5')

In [815]:
outfile = file.path( paste0(result_path, '/03_results/',  model_name) )

In [816]:
outfile

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/example_results//03_results/03_MOFA_MODEL_MI_v1_MOFA.hdf5"

In [817]:
model <- load_model(outfile, verbose = TRUE)

Loading data...

Loading expectations for 2 nodes...

Loading model options...

Loading training options and statistics...

Assigning names to the different dimensions...

Re-ordering factors by their variance explained...

Doing quality control...

Checking views names...

Checking groups names...

Checking samples names...

Checking features names...

Checking dimensions...

Checking there are no features with complete missing values...

Checking sample covariates...

Checking expectations...

Checking for intercept factors...

Warning message in .quality_control(object, verbose = verbose):
“Factor(s) 2, 8 are strongly correlated with the total number of expressed features for at least one of your omics. Such factors appear when there are differences in the total 'levels' between your samples, *sometimes* because of poor normalisation in the preprocessing steps.
”
Checking for highly correlated factors...



In [818]:
### Load the Factor Data of the Model

In [819]:
factors = read.csv(paste0(result_path, '/03_results/', '03_Factor_Data_', mofa_name, '.csv'))

In [820]:
head(factors,2)

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,sample_id
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1.616970,1.922963,-0.04046481,0.6957385,-0.4816992,-0.6951516,0.6601078,-0.1757625,0.04568479,-0.2038762,D1
2,1.749186,0.227296,0.60221439,-0.1048832,-0.7551822,-1.5846769,-0.8970750,-0.1889143,0.22074658,-0.8048890,D2


In [821]:
### Load the Weight Data of the Model

In [822]:
weight_data = read.csv(paste0(result_path, '/03_results/', '03_Weight_Data_', mofa_name, '.csv'))

## Sample Meta Data

In [823]:
## Load the sample meta-data that contains covariates that can be used to group the plot

In [824]:
sample_data = read.csv(paste0(data_path, 'Prepared_Sample_Meta_Data', '.csv'))
sample_data$X = NULL

In [825]:
head(sample_data,2)

,donor_id,disease,sex,development_stage,self_reported_ethnicity,Primary.Genetic.Diagnosis,sample_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,DP2,dilated cardiomyopathy,female,fifth decade human stage,European,PLN,DP2
2,H3,normal,male,sixth decade human stage,Asian,control,H3


# Downstream Analysis of generated model

## Extract and prepare data for plots

### Merge factors and sample data

In [826]:
### Combine the factor and the sample data

In [827]:
head(factors,2)

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,sample_id
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1.616970,1.922963,-0.04046481,0.6957385,-0.4816992,-0.6951516,0.6601078,-0.1757625,0.04568479,-0.2038762,D1
2,1.749186,0.227296,0.60221439,-0.1048832,-0.7551822,-1.5846769,-0.8970750,-0.1889143,0.22074658,-0.8048890,D2


In [828]:
factors_long = melt(factors, id.vars = 'sample_id')

In [829]:
head(factors_long,2)

,sample_id,variable,value
,<chr>,<fct>,<dbl>
1,D1,Factor1,1.616970
2,D2,Factor1,1.749186


In [830]:
merged_data_long = merge(factors_long, sample_data, by.x = 'sample_id', by.y = 'sample_id')

In [831]:
head(merged_data_long,2)

,sample_id,variable,value,donor_id,disease,sex,development_stage,self_reported_ethnicity,Primary.Genetic.Diagnosis
,<chr>,<fct>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,D1,Factor1,1.616970,D1,normal,female,sixth decade human stage,European,control
2,D1,Factor2,1.922963,D1,normal,female,sixth decade human stage,European,control


### Prepare weight data

In [832]:
### Adjust the format of the feature factor weight data to long

In [833]:
feature_weights_long = melt(weight_data, id.vars = c('variable_name', 'type'))

In [834]:
feature_weights_long$view = feature_weights_long$type
feature_weights_long$gene = str_replace(feature_weights_long$variable_name, '.*__', '')

In [835]:
head(feature_weights_long,2)

,variable_name,type,variable,value,view,gene
,<chr>,<chr>,<fct>,<dbl>,<chr>,<chr>
1,adipocyte__ABI1,adipocyte,Factor1,-0.16729734,adipocyte,ABI1
2,adipocyte__ABI2,adipocyte,Factor1,0.04293374,adipocyte,ABI2


### Get top features per factor and for specified threshold

In [836]:
## Get the x% of top features per factor based on the specified threshold (x) in the configuration file

In [837]:
dimensions = unique(feature_weights_long[,c('view', 'variable')])

In [838]:

## Define amont of top genes per fraction 
geneset_oi_pos_per_factor_analyze = feature_weights_long %>% group_by(variable) %>% dplyr::arrange( desc(value),  .by_group = TRUE)  %>% top_frac( as.numeric(top_variable_fraction), value)
geneset_oi_pos_per_factor_analyze$direction = 'positive'

geneset_oi_neg_per_factor_analyze = feature_weights_long %>% group_by(variable) %>% dplyr::arrange(desc(value),  .by_group = TRUE)  %>% top_frac( -as.numeric(top_variable_fraction), value)
geneset_oi_neg_per_factor_analyze$direction = 'negative'

geneset_oi_analyze = rbind(geneset_oi_pos_per_factor_analyze, geneset_oi_neg_per_factor_analyze)
geneset_oi_analyze$fraction = as.numeric(top_variable_fraction)

geneset_oi_per_fraction = geneset_oi_analyze


## Calculate amount of top variables for the selected fraction
amount_geneset_oi_type = geneset_oi_analyze %>% group_by(type, view, variable) %>% dplyr::count()
amount_geneset_oi_type = merge(dimensions, amount_geneset_oi_type, all.x = TRUE)
amount_geneset_oi_type$fraction = as.numeric(top_variable_fraction)


geneset_oi_amounts =amount_geneset_oi_type


In [839]:
## Calculate total amount of features per type
features_per_type = feature_weights_long %>% group_by(type, view, variable) %>% dplyr::count()
colnames(features_per_type) = c('type', 'view', 'variable', 'amount_features')

In [840]:
## Merge top feature amounts and total amounts 
geneset_oi_amounts = merge(  geneset_oi_amounts,features_per_type, all.x = TRUE)

##  Calculate percentages
geneset_oi_amounts$percentage = geneset_oi_amounts$n / geneset_oi_amounts$amount_features

In [841]:
# Set NA when there are no features for this dimension among top %
geneset_oi_amounts[is.na(geneset_oi_amounts)] = 0 

### Prepare heatmap data (based on transformed expression values)

In [842]:
### Subset data on the selected factor (in the configuration)

In [843]:
analysis_data_subset = geneset_oi_per_fraction[geneset_oi_per_fraction$variable %in%  factor_var,]

In [844]:
### Add direction of effect

In [845]:
analysis_data_subset$direction_sign = analysis_data_subset$direction
analysis_data_subset$direction_sign[analysis_data_subset$direction == 'positive'] = '+'
analysis_data_subset$direction_sign[analysis_data_subset$direction == 'negative'] = '-'

In [846]:
head(analysis_data_subset,2)

variable_name,type,variable,value,view,gene,direction,fraction,direction_sign
<chr>,<chr>,<fct>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>
fibroblast.of.cardiac.tissue__NFKBIA,fibroblast.of.cardiac.tissue,Factor2,0.7821156,fibroblast.of.cardiac.tissue,NFKBIA,positive,0.005,+
endothelial.cell__NFKBIA,endothelial.cell,Factor2,0.7455053,endothelial.cell,NFKBIA,positive,0.005,+


In [847]:
### Add cell-type/ type column

In [848]:
analysis_data_subset$cell_type = analysis_data_subset$type

In [849]:
### Concat direction + view for faceting on y -axis

In [850]:
analysis_data_subset$facet =  paste0(analysis_data_subset$direction_sign, analysis_data_subset$view)

In [851]:
head(analysis_data_subset,2)

variable_name,type,variable,value,view,gene,direction,fraction,direction_sign,cell_type,facet
<chr>,<chr>,<fct>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
fibroblast.of.cardiac.tissue__NFKBIA,fibroblast.of.cardiac.tissue,Factor2,0.7821156,fibroblast.of.cardiac.tissue,NFKBIA,positive,0.005,+,fibroblast.of.cardiac.tissue,+fibroblast.of.cardiac.tissue
endothelial.cell__NFKBIA,endothelial.cell,Factor2,0.7455053,endothelial.cell,NFKBIA,positive,0.005,+,endothelial.cell,+endothelial.cell


In [852]:
## Adjust column names in input data

In [853]:
data_long$expression_value = data_long$value
data_long$variable_name = data_long$variable
data_long$variable = NULL
data_long$value = NULL

In [854]:
head(data_long,2)

,sample_id,type,gene,expression_value,variable_name
,<chr>,<chr>,<chr>,<dbl>,<chr>
1,D1,adipocyte,ABI1,-1.1243382,adipocyte__ABI1
2,D2,adipocyte,ABI1,-0.8572543,adipocyte__ABI1


In [855]:
### Merge factor values and expression values

In [856]:
heatmap_data = merge(data_long, analysis_data_subset, by.x = c('variable_name'), by.y = c('variable_name'))

In [857]:
nrow(heatmap_data)

[1] 1060

In [858]:
head(heatmap_data,2)

,variable_name,sample_id,type.x,gene.x,expression_value,type.y,variable,value,view,gene.y,direction,fraction,direction_sign,cell_type,facet
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<fct>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,cardiac.neuron__CST3,H56,cardiac.neuron,CST3,-1.0444088,cardiac.neuron,Factor2,0.6504495,cardiac.neuron,CST3,positive,0.005,+,cardiac.neuron,+cardiac.neuron
2,cardiac.neuron__CST3,H67,cardiac.neuron,CST3,-0.2342192,cardiac.neuron,Factor2,0.6504495,cardiac.neuron,CST3,positive,0.005,+,cardiac.neuron,+cardiac.neuron


In [859]:
### Add sample meta-data

In [860]:
heatmap_data = merge(heatmap_data, sample_data, by.x = 'sample_id', by.y = 'sample_id')

In [861]:
head(heatmap_data,2)

,sample_id,variable_name,type.x,gene.x,expression_value,type.y,variable,value,view,gene.y,⋯,fraction,direction_sign,cell_type,facet,donor_id,disease,sex,development_stage,self_reported_ethnicity,Primary.Genetic.Diagnosis
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<fct>,<dbl>,<chr>,<chr>,⋯,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,D1,endothelial.cell__CDK13,endothelial.cell,CDK13,-2.085356,endothelial.cell,Factor2,-0.6437645,endothelial.cell,CDK13,⋯,0.005,-,endothelial.cell,-endothelial.cell,D1,normal,female,sixth decade human stage,European,control
2,D1,cardiac.neuron__CST3,cardiac.neuron,CST3,1.593219,cardiac.neuron,Factor2,0.6504495,cardiac.neuron,CST3,⋯,0.005,+,cardiac.neuron,+cardiac.neuron,D1,normal,female,sixth decade human stage,European,control


# Heatmap Plot

In [862]:
###Generates heatmap based on configurations

In [863]:
## Specify figure name

In [864]:
figure_name = "FIG05_Heatmap_Feature_Overview"

In [865]:
# Specific Text Descriptions (heatmap) for x and y-axis:
xlabel_heatmap = xlab('Samples') 
ylabel_heatmap = ylab('View')

In [866]:
# Specific Text Descriptions (factor value plot) for x and y-axis:
xlabel_barplot = xlab('Samples') 
ylabel_barplot = ylab('Absolute Factor Value')

In [867]:
## Choose columns for faceting according to parameters specified in config file

In [868]:
## specify faceting for heamtap
heatmap_data$facet2 = heatmap_data[,variable] # x-axis faceting
heatmap_data$facet = paste0(heatmap_data$direction_sign, heatmap_data$view) # y-axis faceting

## specify faceting for factor value plot
analysis_data_subset$facet = paste0(analysis_data_subset$direction_sign,analysis_data_subset$view)
analysis_data_subset$facet2 = 'Absolute Factor Value'

In [869]:
view_var

[1] NA

In [870]:
## filter data on a specific view if specified in the config file
if(view_var %in% unique(heatmap_data$view)){
    heatmap_data = heatmap_data[heatmap_data$view %in% view_var,]
    analysis_data_subset = analysis_data_subset[analysis_data_subset$view %in% view_var,]
    } else{view_var = ''}

In [871]:
head(heatmap_data,2)

,sample_id,variable_name,type.x,gene.x,expression_value,type.y,variable,value,view,gene.y,⋯,direction_sign,cell_type,facet,donor_id,disease,sex,development_stage,self_reported_ethnicity,Primary.Genetic.Diagnosis,facet2
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<fct>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,D1,endothelial.cell__CDK13,endothelial.cell,CDK13,-2.085356,endothelial.cell,Factor2,-0.6437645,endothelial.cell,CDK13,⋯,-,endothelial.cell,-endothelial.cell,D1,normal,female,sixth decade human stage,European,control,normal
2,D1,cardiac.neuron__CST3,cardiac.neuron,CST3,1.593219,cardiac.neuron,Factor2,0.6504495,cardiac.neuron,CST3,⋯,+,cardiac.neuron,+cardiac.neuron,D1,normal,female,sixth decade human stage,European,control,normal


In [872]:
# Plot factor values as barplot

In [873]:
options(repr.plot.width=10, repr.plot.height=25)
factor_value_all_bar = ggplot(data = analysis_data_subset[(analysis_data_subset$variable == factor_var),] , 
                              mapping = aes(x = gene,   y= abs(value)))+
                            plot_config_heatmap_supp  + 
                            geom_bar(stat="identity", width = 0.1) + 
                            coord_flip()+
                            xlabel_barplot +
                            ylabel_barplot +
                            theme( axis.text.y = element_text( hjust = 1, vjust =1))+
                            facet_grid(facet  ~ facet2, scales = 'free', space = 'free', switch = 'y') + theme(strip.background = element_rect(color="black", fill="white", linetype="solid"), strip.placement = "outside", strip.switch.pad.grid = unit(0.5, "cm"))+
                            ggtitle('')


In [874]:
# Example of plot
#factor_value_all_bar

In [875]:
### Generate Heatmap Plot

In [876]:
factor_var

[1] "Factor2"

In [877]:
options(repr.plot.width=width_par, repr.plot.height=height_par)
all_heatmap = ggplot(data = heatmap_data[(heatmap_data$variable == factor_var) ,], # TBD whether to filter on sample subset
                     mapping = aes(sample_id, gene.x,  fill= expression_value)) +
    scale_fill_gradient2(low="#1D2ED8", high="#D8911D", midpoint = 0) + 
    geom_tile() + 
    xlabel_heatmap +
    ylabel_heatmap +
    plot_config_heatmap_supp  + theme(axis.text.x = element_blank()) + 
    facet_grid(facet  ~facet2, scales = 'free', space = 'free',  switch = "y") + theme(strip.background = element_rect(color="black", fill="white", linetype="solid"), strip.placement = "outside", strip.switch.pad.grid = unit(0.4, "cm"), strip.text.y.left = element_text(angle=0))+
    theme( axis.text.y = element_text( hjust = 1, vjust =1)) + 
    ggtitle(paste0('Normalized Expression Values of top ',as.numeric(top_variable_fraction) *100 *2, '% features'))

    

In [878]:
# Example heatmap plot
#all_heatmap

In [879]:
### Combined factor and heatamp plots and save the plot

In [880]:
legend =  get_legend(all_heatmap)

In [881]:
options(repr.plot.width=width_par, repr.plot.height=height_par)
combined = ggarrange(all_heatmap + theme(legend.position = 'none'),
         factor_value_all_bar + 
          theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank(), strip.background.y = element_blank() , strip.text.y = element_blank()),  
         nrow=1, widths = c(4,1), align = 'h')
combined = annotate_figure(combined, right = legend)

In [882]:
pdf(paste0('figures/05_figures/', figure_name,view_var, '_', factor_var, '_',  top_variable_fraction,  mofa_name, '.pdf'), width =width_par, height =height_par)
print(combined)
dev.off()

png 
  2